In [1]:
%matplotlib notebook 

import matplotlib.pyplot as plt
import numpy as np

from scipy.constants import e, m_e, c, pi, epsilon_0, mu_0
from tqdm.auto import tqdm
from unwrap import unwrap

from axiprop.lib import PropagatorResamplingFresnel
from axiprop.lib import PropagatorResamplingPlasma
from axiprop.lib import PropagatorSymmerticPlasma

from axiprop.common import ScalarField
from axiprop.utils import mirror_parabolic

from axiprop.plasma import get_plasma_ionized
from axiprop.plasma import make_ADK
from axiprop.plasma import get_plasma_OFI

from LPICS.ics import CheatSheet

r_e = e**2 / (4 * np.pi * epsilon_0 * m_e * c**2)

In [2]:
from copy import deepcopy

mycmap = deepcopy(plt.cm.Greys)
mycmap._init()
filt = np.r_[0:1:32j]
mycmap._lut[-filt.size:, 0] += filt

myReds = deepcopy(plt.cm.Reds)
myReds._init()
myReds._lut[:, -1] = np.sin(np.r_[0:np.pi/2:259j])**4

myGreys = deepcopy(plt.cm.Greys)
myGreys._init()
myGreys._lut[:, -1] = np.sin(np.r_[0:np.pi/2:259j])**4

def plot_colored(x, y, z, cmap=plt.cm.jet, steps=10, **kw_args):
    z = np.asarray(z)
    z -= z.min()
    z /= z.max()
    it=0
    while it<z.size-steps:
        x_segm = x[it:it+steps+1]
        y_segm = y[it:it+steps+1]
        c_segm = cmap( z[it+steps//2] )
        plt.plot(x_segm, y_segm, c=c_segm, **kw_args)
        it += steps

In [3]:
# Laser init
lambda0 = 0.8e-6
tau_fwhm = 28e-15
R_las = 38.1e-3
check_laser = True
Intensity = 5e17
a0 = 0.5

# Mirror
R_mirr = R_las
f_N = 40                 # f-number f/40 
f0 = 2 * R_mirr * f_N    # focal length
w0 = 2 / np.pi * lambda0 * f_N

L_R = np.pi * w0**2 / lambda0

# Plasma
n_gas = 4e17 * 1e6
my_element = 'H'

channel_coef = 1./( np.pi * r_e * w0**2 * n_gas )
channel_pow = 2

k_p = (e**2 * n_gas / m_e / epsilon_0)**0.5 / c
kp2 = k_p**2

pack_ADK, Uion = make_ADK(my_element)

def dens_func(z, r):
    r_profile = 1. + channel_coef * np.abs(r / w0)**channel_pow
    r_profile *= np.exp( -( r / (5 * w0) )**4 )
    r_profile += 1 - np.exp(- ( r / (5 * w0) )**4 )
    r_peak = r[r_profile == r_profile.max()].mean()
    r_profile[r>r_peak] = r_profile.max()

    z_profile = (z>=f0) + np.exp( -np.abs(z - f0)**2 / L_R**2 ) * (z<f0)
    
    return r_profile * z_profile

# Numeric parameters
tau = tau_fwhm / (2*np.log(2))**0.5
k0 = 2 * np.pi / lambda0

L_kz = 16 / (c*tau)

Rmax = 3.5 * R_las
Nr = 512

Rmax_new = 600e-6
Nr_new = 1024
Nkr_new = Nr_new

laser_cs = CheatSheet(Intensity=Intensity,
                      #a0=a0, 
                      w0=w0*1e6,
                      tau_fwhm=tau_fwhm * 1e15,
                      )

EnergyLaser = laser_cs.prm['Energy']
a0 = laser_cs.prm['a0']
a0_far = a0 / (1 + (f0/L_R)**2)**0.5
n_c = 1e6 * laser_cs.prm['n_c']

In [4]:
dt = lambda0 / c / 32

Fields = {}
field_names = ['E_init', 'E_foc', 'A_foc', 'J_plasma']

for field in field_names:
    Fields[field] = ScalarField(k0, L_kz, (-6*tau, 6*tau), dt, n_dump=32)

k_freq = Fields['E_init'].k_freq

prop_plasma = PropagatorSymmerticPlasma(
       r_axis=(Rmax_new, Nr_new),
       kz_axis=k_freq,
       backend='CU',
)

prop_far = PropagatorResamplingFresnel(
    r_axis=(Rmax, Nr),
    kz_axis=k_freq,
    r_axis_new=prop_plasma.r,
    Nkr_new=Nkr_new,
    N_pad=8,
    backend='CU',
)

Available backends are: NP, CL, CU, NP_FFTW
CU is chosen
Available backends are: NP, CL, CU, NP_FFTW
CU is chosen


In [24]:
# distance from mirror to plasma
z_0 = f0 - 4 * L_R

N_steps = 400

# Propagation axis in plasma
z_axis = z_0 + np.r_[0 : 14 * L_R : 1j * N_steps]
dz_step = np.gradient(z_axis)

# Making the laser
Fields['E_init'].make_gaussian_pulse(a0_far, tau, prop_far.r, R_las, n_ord=2 )
E_far = Fields['E_init'].Field_ft.copy()

# Adding the mirror
E_far *= mirror_parabolic( f0, prop_far.kz, prop_far.r )

# Propagate field to the plasma
E_near = prop_far.step(E_far, z_0)

# Initiate plasma propagator
E_near = Fields['E_foc'].apply_boundary_ft(E_near)
prop_plasma.initiate_stepping(E_near)
fields_sync = True

# Initiate diagnostics
dn_diags = 2
E_onax = []
I_on_r = []
W_laser = []

CurrentField = 'Motion'

In [25]:
# Main loop
for iz in tqdm(range(z_axis.size)):
    t_loc = (z_0 + prop_plasma.z_propagation) / c
    n_pe0 = n_gas * dens_func( z_axis[iz], prop_plasma.r_new )

    Fields['E_foc'].import_field_ft(
        E_near, t_loc, r=prop_plasma.r_new,
        transform=False)

    fields_sync = False
    
    if np.mod(iz, dn_diags)==0:
        E_onax.append( Fields['E_foc'].get_temporal_slice() )
        I_on_r.append( np.abs(Fields['E_foc'].Field_ft**2).sum(0) )
        W_laser.append( Fields['E_foc'].Energy_ft )
        
    if CurrentField=='Simple':
        kp2_ax = (e**2 * n_pe0 / m_e / epsilon_0) / c**2
        kp2 = kp2_ax[0]

        Fields['J_plasma'].import_field_ft(
            1j * kp2_ax[None,:] * E_near / (Fields['E_foc'].omega[:,None] * mu_0),
            t_loc, r=prop_plasma.r_new, transform=False)

    if CurrentField=='Motion':
        if not fields_sync:
            Fields['E_foc'].frequency_to_time()

        A_near = 1j *  E_near / Fields['E_foc'].omega[:,None]

        Fields['A_foc'].import_field_ft(
            A_near, t_loc, r=prop_plasma.r_new
        )
        
        J_plasma, n_e, T_e = get_plasma_ionized(
            Fields['E_foc'].Field, Fields['A_foc'].Field, 
            dt, n_pe0
        )

        Fields['J_plasma'].import_field(
            J_plasma, t_loc, r=prop_plasma.r_new,
        )
        
        kp2 = (e**2 * n_e[0] / m_e / epsilon_0) / c**2
        
    if CurrentField=='Ionization':
        if not fields_sync:
            Fields['E_foc'].frequency_to_time()

        A_near = 1j *  E_near / Fields['E_foc'].omega[:,None]

        Fields['A_foc'].import_field_ft(
            A_near, t_loc, r=prop_plasma.r_new
        )

        J_plasma, n_e, T_e = get_plasma_OFI(
            Fields['E_foc'].Field, Fields['A_foc'].Field, 
            dt, n_pe0, pack_ADK, Uion, Z_init=0
        )

        Fields['J_plasma'].import_field(
            J_plasma, t_loc, r=prop_plasma.r_new,
        )

        kp2 = (e**2 * n_e[0] / m_e / epsilon_0) / c**2  

    E_near = prop_plasma.stepping_step_withJ(
        kp2, Fields['J_plasma'].Field_ft,
        dz_step[iz], u_out=E_near, conservative=True
    )  
    
    E_near = Fields['E_foc'].apply_boundary_ft(E_near)
    
t_loc = (z_0 + prop_plasma.z_propagation) / c
Fields['E_foc'].import_field_ft(E_near, t_loc, 
                                r=prop_plasma.r_new)

E_onax.append( Fields['E_foc'].get_temporal_slice() )
I_on_r.append( (Fields['E_foc'].Field_ft**2).sum(0) )
W_laser.append( Fields['E_foc'].Energy_ft )

E_onax = np.array(E_onax)
I_on_r = np.array(I_on_r)
W_laser = np.array(W_laser)

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
plt.figure()
plt.plot( (W_laser-EnergyLaser)/EnergyLaser * 100 )

plt.figure()
ext = np.array([Fields['E_foc'].t.min(), Fields['E_foc'].t.max(),
                (z_axis-f0).min(), (z_axis-f0).max() ])

ext[:2] *= 1e15
ext[2:] /= L_R

plt.imshow(e*np.abs(E_onax) / (m_e*c**2*k0), 
           origin='lower', aspect='auto', 
           extent=ext, #interpolation='none',
           cmap=plt.cm.nipy_spectral,
          )

dens_slice = dens_func(z_axis, np.array([0]))

v_g = 1.0 - np.sqrt( 1.0 + n_gas * dens_slice / n_c )
Gouy_shift = np.arctan((z_axis-f0)/L_R)/ (k0*c) 
Gouy_shift -= Gouy_shift[0]

plt.plot( (v_g * (z_axis-z_axis[0])/c - Gouy_shift)* 1e15 , (z_axis-f0)/L_R )

plt.colorbar()

plt.figure()
ext = np.array([(z_axis-f0).min(), (z_axis-f0).max(),
                Fields['E_foc'].r.min(), Fields['E_foc'].r.max(),
               ])
ext[:2] *= 1e3
ext[2:] *= 1e6

plt.imshow(np.log10(np.abs(I_on_r)).T, 
           origin='lower', aspect='auto', 
           extent=ext, 
           cmap=plt.cm.nipy_spectral,
           vmin=np.log10(np.abs(I_on_r)).max()-17,
          )

plt.colorbar()

plt.figure()
ext = np.array([(z_axis-f0).min(), (z_axis-f0).max(),
                prop_plasma.r_new.min(), prop_plasma.r_new.max(),
               ])

ext[:2] *= 1e3
ext[2:] *= 1e6

plt.imshow(dens_func(z_axis[:, None], prop_plasma.r_new[None,:]).T,
           aspect='auto', origin='lower',
           extent=ext, 
          )

In [ ]:
plt.figure()
ext = np.array([(z_axis-f0).min(), (z_axis-f0).max(),
                Fields['E_foc'].r.min(), Fields['E_foc'].r.max(),
               ])

ext[:2] /= L_R
ext[2:] /= w0

val = (np.abs(I_on_r))**.5
val /= val.max()

plt.imshow(val.T, 
           origin='lower', aspect='auto', 
           extent=ext, 
           cmap=plt.cm.afmhot_r
          )
#plt.colorbar()

dens2D = dens_func(z_axis[:, None], prop_plasma.r_new[None,:])

plt.contourf( dens2D.T,
             extent=ext, cmap=myGreys, 
             levels=16, vmax=1.3 * dens2D.max(),
            )

plt.ylim(0, 7)
plt.xlim(-4, )
plt.xlabel('$z/L_R$', size=14)
plt.ylabel('$r/w_0$', size=14)

In [ ]:
field_name = 'E_foc'

ext = np.array(
    [
        Fields[field_name].k_freq.min()/k0, 
        Fields[field_name].k_freq.max()/k0, 
        0, Fields[field_name].r.max()*1e6
    ]
)


val = np.abs(Fields[field_name].Field_ft)
plt.figure()
plt.imshow((val).T, 
           origin='lower', aspect='auto', 
           extent=ext, 
           cmap=plt.cm.seismic,
           vmin=np.log10(val.max())-15
          )

plt.ylim(0, 100)
plt.xlim(0.8, 1.2)


plt.colorbar()

val = unwrap(np.angle(Fields[field_name].Field_ft))

plt.figure()
plt.imshow(val.T, 
           origin='lower', aspect='auto', 
           extent=ext, 
           cmap=plt.cm.seismic,
           vmin=-8*np.pi, vmax=8*np.pi,
          )

#plt.ylim(0, 100)
#plt.xlim(0.8, 1.2)
plt.colorbar()